In [1]:
import os
import logging
from glob import glob
from datetime import datetime
from collections import namedtuple
from pickle import dump, load

import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import transformers
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer, EarlyStoppingCallback, BatchEncoding
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer, TrainingArguments
from transformers.integrations import TensorBoardCallback
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
import optuna

from arabert.preprocess import ArabertPreprocessor
from utilities import *

In [2]:
# Config
device = torch.device("cuda")
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
logging.disable(logging.WARNING)

# Model params
# model_name = "aubmindlab/bert-base-arabertv2"
model_name = "aubmindlab/bert-large-arabertv2"
    
# Preprocessing params
sequence_length = 128
arabert_prep = ArabertPreprocessor(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Data param
df = get_music_df().sample(frac=0.01)
model_path = "models/finalized_models/2021-09-30-train-0.8921535648994515"

# Preprocessing & loading model

In [3]:
%%time

df["Labels"] = df["Region"].apply({
    "EGY": 0,
    "GLF": 1,
    "IRQ": 2,
    "LEV": 3,
    "NOR": 4
  }.get)

df["Text"] = df["Text"].apply(arabert_prep.preprocess)
df_encoding = tokenize(tokenizer, df["Text"].to_list(), sequence_length)

test_set = Dialect_dataset(df_encoding, df["Labels"].to_list())
model = AutoModelForSequenceClassification.from_pretrained(model_path)

Wall time: 1.61 s


# Testing

In [4]:
trainer = Trainer(model=model,compute_metrics=compute_metrics)
prediction = trainer.predict(test_set)
prediction.metrics

 94%|█████████▍| 50/53 [00:01<00:00, 33.44it/s]D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 2.0783586502075195,
 'test_macro_f1': 0.4021618029746552,
 'test_macro_precision': 0.41860859228578595,
 'test_macro_recall': 0.4327142857142857,
 'test_accuracy': 0.5141509433962265,
 'test_runtime': 3.7345,
 'test_samples_per_second': 113.536,
 'test_steps_per_second': 14.192}

In [5]:
y_pred = [np.argmax(pred) for pred in prediction[0]]
y_true = df["Labels"]
accuracy_score(y_true , y_pred)

0.5141509433962265

In [6]:
text = "والله مابعرف اغلب الناس رواتبها ومش ععارفين كيف ممشين هاذ بوخذ الاف وعدا عن السيارات والبدل والسفرات والخ وبحكيلك قليل هو الفساد بيجي من قليل" 
predict_dialect(model_path, text, tokenizer, arabert_prep.preprocess, sequence_length)

100%|██████████| 1/1 [00:00<00:00, 334.23it/s]


'LEV'